## **Hungarian Algorithm** - Step by Step Process and Development


## Introduction 
Out goal is to implement in Python the Hungarian Algorithm solving the **assignment problem** where: 
* We have n workers and n tasks 
* Each worker-task assignment has a cost. 
* We want to find the optimal assignment to minimize the total cost.

## Step 0 - **The Input**

We are given a **cost matrix**, and each element `c[i][j]` is the cost of assigning the task 'j' to the worker 'i'. We aspire to find the one-to-one assignments that will be able to minimize the total cost.

In [1]:
''' EXAMPLE OF COST MATRIX'''
cost_matrix = [
    [4,1,3], 
    [2,0,5], 
    [3,2,2]
]

Our objective is to find the combination of assignments that, then summed the individual costs, minimizes the total cost.

## Step 1 - **Row Reduction**
The first step is to substract the minimun value of each row from every element in that row. We do this because in the end, it doesn't change the optimal assignment and it guarantees at least one zero in each row, which simplifies finding potential assignments.

In [2]:
import numpy as np  # for better treatment of matrix elements

C = np.array([
    [4,1,3], 
    [2,0,5], 
    [3,2,2]
])

'''Step 1: Substract the row minimus'''
# np.min is a function across COLUMNS, since axis = 1, row-wise;
# this gives a 1D array with the minimun for each row.
row_min = np.min(C, axis=1) 
print('Row Min:')
print(row_min)

# row_min[:, np.newaxis] transforms row_min from shape (3, 0) into shape (3,1)
# We do this because we want to subtract row-wise. We need row_min to align with C
# dimensions so broadcasting works correctly.
C_row_reduced = C - row_min[:, np.newaxis]
print('C row reduced:')
print(C_row_reduced)

Row Min:
[1 0 2]
C row reduced:
[[3 0 2]
 [2 0 5]
 [1 0 0]]


## Step 2 - **Column Reduction**
The goal for each column is to find the smallest element, and substract it from all elements in that column. 

In [3]:
# axis = 0 -> apply column wise
col_min = np.min(C_row_reduced, axis=0)
print('Min columns: \n', col_min)

Min columns: 
 [1 0 0]


Now we can substract these column minimus in the corresponding columns. 

In [4]:
C_reduced = C_row_reduced - col_min
print('C reduced: \n', C_reduced)

C reduced: 
 [[2 0 2]
 [1 0 5]
 [0 0 0]]


## Step 3 - **Cover all zeros with minimun number of lines**
After all the reductions, we find some zeros in our cost matrix. We want to cover (cross out) all zeros with the **minimun number of horizontal and vertical lines**. 

We do this because if we can cover all zeros with exactly 'n' lines ('n' being the size of the matrix), then we can assign directly. If not, we need to modify the matrix further (step 4). There are several ways to do this step manually or algorithmically.

### 1. Mark independent zeros (preliminary step)
Find a zero, and if there is not other zeros in its row or column, mark it as **starred** with a \*. Repeat this until all zeros are starred or unmarked. If we do this manually, we would obtain: 

````
[
    [ , *, ], 
    [ , *, ], 
    [*, *, *] --> we have a conflict here
]
````
In practice, we only star one zero per row and per column. Since we cannot have multiple stars in the same row/column, we must select a set of independent zeros.

### 2. Cover columns containing starred zeros
After starring, cover all columns that contain a starred zero and count how many were used. If we can cover all zeros with exactly n lines we are done, if not, we go to step 4.

### Implementation
We need to write: 
* A method to find the independent zeros
* A method to cover columns with starred zeros
* Count how many lines we used.

In [5]:
n = C_reduced.shape[0] # we save the size of the matrix

''' 
Step 3.1. Star independent zeros
Initilize mask matrix: 0 = no mark; 1 = starred; 2 = primed
'''
mask = np.zeros_like(C_reduced, dtype=int)
row_covered = np.zeros(n, dtype=bool)
col_covered = np.zeros(n, dtype=bool)

# Find independent zeros and star them 
for i in range(n): 
    for j in range(n): 
        if C_reduced[i,j] == 0 and not row_covered[i] and not row_covered[j]: 
            mask[i,j] = 1 # star the zero 
            row_covered[i] = True
            col_covered[j] = True 
            
# reset the row and column covers for the next step 
row_covered[:] = False
col_covered[:] = False 

print('Mask matrix after starring the independent zeros: \n', mask)

Mask matrix after starring the independent zeros: 
 [[0 1 0]
 [0 1 0]
 [0 0 1]]


In [6]:
''' 
Step 3.2. Cover the columns containing the starred zeros 
'''
for j in range(n): 
    if np.any(mask[:,j] == 1): 
        col_covered[j] = True 

print('Columns covered after initial starring: ')
print(col_covered)

Columns covered after initial starring: 
[False  True  True]


In [7]:
''' 
Step 3.3. Count number of covered columns 
'''
num_lines = np.sum(col_covered)
print(f'Number of lines used to cover all the starred zeros: {num_lines}')

Number of lines used to cover all the starred zeros: 2


Since n = 3 > 2 = lines used to cover all the starred zeros, we proceed with the step 4.

## Step 4 - **Adjusting the Matrix** (create more zeros)

We need to modify the matrix to create additional zeros, but carefully, so we don't ruin previous work. The goal of step 4 is to modify the matrix to create more zeros without affecting the positions of existing starred zeros. 

We can modify the matrix following these rules: 
1. Find the smallest value among all uncovered element (we call this value `min_uncovered`)
2. Substract the `min_uncovered` from every uncovered element. 
3. Add `min_uncovered` to every element that is covered **twice** (both row and column covered). 
4. Leave elements covered only once (either row or column) unchanged.

This works because substracting from the uncoveres elements creates new zeros. Adding to double-covered elements preserves the previous zeros. It ensures that the feasibility of already starred zeros is not affected.

In [8]:
'''Step 4 - Modify the matrix to create more zeros'''

# 1. Find minimun uncovered value
uncovered_values = []
for i in range(n): 
    for j in range(n): 
        if not row_covered[i] and not col_covered[j]: 
            uncovered_values.append(C_reduced[i,j])

if uncovered_values: 
    min_uncovered = min(uncovered_values)
else: 
    min_uncovered = 0 # safety check if no uncovered values

print(f'Minimun uncovered value: {min_uncovered}')

# 2. Substract the min_uncovered from uncovered elements
# 3. Add min_uncovered to all elements covered twice (both row and column)

for i in range(n): 
    for j in range(n): 
        if not row_covered[i] and not row_covered[j]: 
            C_reduced[i,j] -= min_uncovered
        elif row_covered[i] and col_covered[j]: 
            C_reduced[i,j] += min_uncovered

print('Matrix after adjustment')
print(C_reduced)

Minimun uncovered value: 0
Matrix after adjustment
[[2 0 2]
 [1 0 5]
 [0 0 0]]


Once we have this, we go back to step 3 and repeat the process. The cycle repeats until the number of lines equals n, at which point we're ready for the assignment.

## Step 5 - **Extract the Optimal Assignment**
The goal is from the final matrix - where we covered all zeros using n lines - we now extract the **optimal assignment**, which is a pairing of workers to task. 

We will use de **starred zeros** to decide the final assignments. Each starred zero represents an assignment (worker i to task j). We need to select exactly one zero per row and per column - this is ensured by the previous steps. 

### Assignment Rules
From the mask matrix, for each row we will find the column where `mask[i][j] == 1`; that column represents the task assigned to worker i. 

In [14]:
''' 
STEP 5 - Extract the assignment from the final mask
'''
assignment = []

# simulate a valid mask
mask = [
 [0, 1, 0],
 [1, 0, 0],
 [0, 0, 1]
]


for i in range(n): # for each row 
    for j in range(n): # for each column 
        if mask[i][j] == 1: # starred zero --> assigned
            assignment.append((i,j)) # add the tuple
            break # move on to the next row 

print("Optimal assignment (worker->task):")
for worker, task in assignment: 
    print(f"Worker: {worker} -> Task: {task}")

Optimal assignment (worker->task):
Worker: 0 -> Task: 1
Worker: 1 -> Task: 0
Worker: 2 -> Task: 2


Now we can compute the total cost of the assignment by summing the selected entries from the **original cost matrix** (not the reduced one): 

In [16]:
total_cost = sum(C[i,j] for i,j in assignment)
print(f"Total cost of assignment: {total_cost}")

Total cost of assignment: 5
